#ServerFile

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install diffusers transformers accelerate safetensors torch torchvision
!mkdir images

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 130.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfull

In [2]:
!pip install fastapi uvicorn

In [3]:
from pydantic import BaseModel
from diffusers import StableDiffusionXLPipeline
import torch
import os, shutil
from PIL import Image
from IPython.display import display
from fastapi import FastAPI, File, UploadFile, Form
from random import randint
import datetime


In [4]:
%%writefile main.py
from pydantic import BaseModel
from diffusers import StableDiffusionXLImg2ImgPipeline
import torch
import os, shutil
from PIL import Image
from fastapi import FastAPI, File, UploadFile, Form
from random import randint
import datetime

from fastapi.staticfiles import StaticFiles

app = FastAPI()
app.mount("/images", StaticFiles(directory="images"), name="images")


# Load Model
my_model_path_in_drive = ""

pipe_img = StableDiffusionXLImg2ImgPipeline.from_single_file(
    my_model_path_in_drive,
    torch_dtype=torch.float16,
    variant="fp16"
).to("cuda")

@app.post("/generar")
async def generar_img(
    prompt: str = Form(...),
    negative_prompt: str = Form(""),
    strength: float = Form(0.75),
    image: UploadFile = File(...)
):
    #  UploadFile
    init_image = Image.open(image.file).convert("RGB").resize((1024, 1024))

    # gen img
    result = pipe_img(
        prompt=prompt,
        negative_prompt=negative_prompt,
        image=init_image,
        strength=strength,
        guidance_scale=5,
        num_inference_steps=40
    ).images[0]

    date = datetime.datetime.now()
    date_str = date.strftime("%Y-%m-%d_%H-%M-%S")
    image_name = f"{randint(0, 100)}_{date_str}.png"
    os.makedirs("images", exist_ok=True)
    image_path = os.path.join("images", image_name)
    result.save(image_path)

    return {"mensaje": "Imagen generada", "file_url": image_name}


Writing main.py


In [5]:
!uvicorn main:app --reload

INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [7147] using StatReload
2025-07-24 21:20:38.997123: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753392039.016902    7149 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753392039.022924    7149 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Fetching 17 files:   0% 0/17 [00:00<?, ?it/s]
merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json: 100% 472/472 [00:00<00:00, 5.12MB/s]


config.json: 100% 575/575 [00:00<00:00, 5.88MB/s]


model_index.json: 100% 609/609 [00:00<00:

**configure ngrok tunnel**

open the google colab terminal and run:

curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc \
  | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null \
  && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
  | sudo tee /etc/apt/sources.list.d/ngrok.list \
  && sudo apt update \
  && sudo apt install ngrok

ngrok config add-authtoken *Your Token*

ngrok http http://localhost:8000

*

In [ ]:
"""
pipe_img = StableDiffusionXLImg2ImgPipeline.from_single_file(
    '',
    torch_dtype=torch.float16,
    variant="fp16"
).to("cuda")
"""


In [ ]:
"""
pipe = StableDiffusionXLPipeline.from_single_file(
    ''
    torch_dtype=torch.float16,
    variant="fp16"
).to("cuda")

"""


Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]